Classificação de Textos usando BERT

# Instalação das bibliotecas necessárias


In [ ]:
# Descomente e execute se necessário
#!pip install transformers pandas numpy scikit-learn torch datasets

In [ ]:
!pip install datasets

# Importação das bibliotecas


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Verificar se GPU está disponível


In [ ]:
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando {dispositivo}")

# Baixar e preparar o dataset B2W-Reviews01

https://github.com/americanas-tech/b2w-reviews01/tree/main

In [ ]:
print("Baixando o dataset B2W-Reviews01...")
url = "https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv"
df = pd.read_csv(url, low_memory=False)

In [ ]:
df.head()

# Preparar os dados - apenas textos e sentimentos


In [ ]:
df = df[['review_text', 'overall_rating']]
df.columns = ['texto', 'nota']

In [ ]:
df.head(10)

## Converter notas para sentimento binário (0-negativo, 1-positivo)


In [ ]:
df = df[(df['nota'] <= 2) | (df['nota'] >= 4)].copy()

In [ ]:
df['sentimento'] = df['nota'].apply(lambda x: 1 if x >= 4 else 0)

In [ ]:
df = df[['texto', 'sentimento']]

In [ ]:
df = df[df['texto'].notna() & (df['texto'] != '')].reset_index(drop=True)

In [ ]:
df.head(10)

## Verificar distribuição


In [ ]:
print(f"Total de exemplos: {len(df)}")
print(f"Exemplos positivos: {sum(df['sentimento'])}")
print(f"Exemplos negativos: {len(df) - sum(df['sentimento'])}")

## Limitar dataset para processamento mais rápido


In [ ]:
# Usando apenas 1000 exemplos balanceados para demonstração
neg = df[df['sentimento'] == 0].sample(500, random_state=42)
pos = df[df['sentimento'] == 1].sample(500, random_state=42)
df = pd.concat([neg, pos]).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df.shape

## Separar em treino e teste

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


# Tokenização

## Carregar o Tokenizador do modelo BERT pré-treinado para português

In [ ]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Função de pré-processamento para tokenização


In [ ]:
exemplo = train_df.iloc[0]["texto"]
exemplo

In [ ]:
tokenizer(exemplo, truncation=True, padding="max_length", max_length=128)

In [ ]:
def preprocess_function(examples):
    # Tokenizar os textos e definir truncation/padding
    tokenized = tokenizer(
        examples["texto"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # IMPORTANTE: Adicionar as etiquetas (labels)
    tokenized["labels"] = examples["sentimento"]

    return tokenized

In [ ]:
preprocess_function(train_df.iloc[0])

## Converter DataFrames para o formato compatível com o Pytorch


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_dataset

## Aplicar tokenização

In [ ]:
train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

In [ ]:
train_tokenized

# Preparar o modelo para treinamento

## Carregar modelo pretreinado e preparado para classificação

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    device_map="auto"
)

In [ ]:
model

## Similaridade com embeddings do BERT

In [ ]:
texto = "Este produto é excelente e durável."

# Tokenizar o texto
entradas = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Mover inputs para o mesmo dispositivo que o modelo
entradas = {k: v.to(dispositivo) for k, v in entradas.items()}

# Obter embeddings (representação vetorial) da frase
with torch.no_grad():
    outputs = model(**entradas, output_hidden_states=True)


In [ ]:
outputs

In [ ]:
outputs.keys()

logits é a saida de classificacão... que não treinamos ainda e não fazem sentido agora

In [ ]:
outputs.logits

hidden_states são as representações internas que o modelo constrói

In [ ]:
outputs.hidden_states[-1].shape

In [ ]:
def obter_embedding(texto):
    # Tokenizar o texto
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

    # Mover inputs para o mesmo dispositivo que o modelo
    inputs = {k: v.to(dispositivo) for k, v in inputs.items()}

    # Obter embeddings (representação vetorial) da frase
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Usar a representação da camada de saída [CLS]
    # O token [CLS] (primeiro token) contém a representação da frase inteira
    embedding = outputs.hidden_states[-1][0, 0, :].cpu().numpy()
    return embedding

In [ ]:
def calcular_similaridade(frase1, frase2):
    # Obter embeddings para cada frase
    emb1 = obter_embedding(frase1)
    emb2 = obter_embedding(frase2)

    # Normalizar os vetores
    emb1_norm = emb1 / np.linalg.norm(emb1)
    emb2_norm = emb2 / np.linalg.norm(emb2)

    # Calcular similaridade de cosseno
    similaridade = np.dot(emb1_norm, emb2_norm)
    return similaridade

In [ ]:
frase1= "Este produto é excelente e durável."
frase2= "Este produto é ruim e ineficiente."

calcular_similaridade(frase1, frase2)

In [ ]:
frase1= "Este produto é excelente e durável."
frase2= "O produto tem ótima qualidade e é resistente."

calcular_similaridade(frase1, frase2)

In [ ]:
frase1= "Este produto é excelente e durável."
frase2= "Subi num ônibus em madagascar"

calcular_similaridade(frase1, frase2)

# Treinamento

## Configurar argumentos de treinamento


In [ ]:
training_args = TrainingArguments(
    output_dir="./resultados",
    num_train_epochs=1,            # Apenas 1 época para demonstração
    #per_device_train_batch_size=8,
    #learning_rate=2e-5,
)

## Função para cálculo das métricas

In [ ]:
def compute_metrics(resultado):
    rotulos_verdadeiros = resultado.label_ids
    rotulos_preditos = resultado.predictions.argmax(-1)
    acc = (rotulos_preditos == rotulos_verdadeiros).mean()
    return {"accuracy": acc}

## Configurar o treinador


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

## Treinar o modelo


In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
print("\nIniciando treinamento...")
trainer.train()

# Avaliar o modelo


In [ ]:
print("\nAvaliando o modelo...")
eval_results = trainer.evaluate()
print(f"Acurácia: {eval_results['eval_accuracy']:.4f}")

## Função para classificar novos textos


In [ ]:
def classificar_texto(texto):
    # Tokenizar e preparar o texto para o modelo
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

    # Mover inputs para o mesmo dispositivo que o modelo
    inputs = {k: v.to(dispositivo) for k, v in inputs.items()}

    # Fazer a predição
    with torch.no_grad():
        outputs = model(**inputs)

    # Obter a classe prevista (0=Negativo, 1=Positivo)
    sentimento = "Positivo" if outputs.logits.argmax().item() == 1 else "Negativo"

    return sentimento

## Testar com novos exemplos

In [ ]:
texto = "Produto excelente, entrega rápida e atendimento nota 10!"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")

In [ ]:
texto = "Péssima experiência, o produto veio quebrado e o suporte não resolveu."

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")

In [ ]:
texto = "eu recomendaria"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")

In [ ]:
texto = "eu desrecomendaria"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")

In [ ]:
texto = "normal, não supreendeu."

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")

# Ajustes para o github

In [ ]:
pip install nbstripout


In [ ]:
!nbstripout classificacao_de_textos_com_bert.ipynb